In [1]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
import hdf5getters as GETTERS
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.35:7077") \
        .appName("Jenny")\
        .config("spark.executor.cores",2)\
        .config("spark.dynamicAllocation.enabled", False) \
        .config("spark.shuffle.service.enabled", False) \
        .getOrCreate()
        
sc = spark_session.sparkContext

In [2]:
msd_subset_path='/home/ubuntu/MillionSongSubset/'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
msd_code_path='/home/ubuntu/MSongsDB'
dbfile = '/home/ubuntu/MSongsDB/Tasks_Demos/SQLite/track_metadata.db'
sys.path.append(os.path.join(msd_code_path,'PythonSrc'))

def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    cnt = 0
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        cnt += len(files)
        for f in files :
            func(f)       
    return cnt

print('number of song files:',apply_to_all_files(msd_subset_data_path))

number of song files: 10000


In [3]:
all_the_info = set()

def func_to_get_all_the_info(filename):
    h5 = GETTERS.open_h5_file_read(filename)
    #track_id = GETTERS.get_track_id(h5)
    artist = GETTERS.open_artist_name(h5)
    year = GETTERS.get_year(h5)
    #danceability = GETTERS.get_danceability(h5)
    #energy = GETTERS.get_energy(h5)
    loudness = GETTERS.get_loudness(h5)
    hotness = GETTERS.get_song_hotttnesss(h5)
    pitch = GETTERS.get_key(h5)
    all_the_info.add((year, loudness, hotness))
    h5.close()

apply_to_all_files(msd_subset_data_path,func=func_to_get_all_the_info)

print('found',len(all_the_info),'tracks')
for k in range(5):
    print(list(all_the_info)[k])

found 9975 tracks
(1999, -15.008, 0.34092275586437804)
(0, -15.81, 0.24099189058358852)
(0, -5.129, nan)
(1989, -10.062, nan)
(0, -3.909, 0.0)


In [4]:
rdd = sc.parallelize(list(all_the_info))
print(rdd.take(1))

[(1999, -15.008, 0.34092275586437804)]


In [5]:
sc.stop()